In [1]:
import sys
import pymol
import numpy as np
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

# load something into the PyMOL window
from pymol import cmd

CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'

Gtk-Message: 13:23:23.347: Failed to load module "pk-gtk-module"
Cannot open file '/home/vit/Projects/cryptoshow-analysis/src/D-visualize/data/pymol/icons/icon2.svg', because: No such file or directory
Cannot open file '/home/vit/Projects/cryptoshow-analysis/src/D-visualize/data/pymol/icons/icon2.svg', because: No such file or directory


In [ ]:
sys.path.append('/home/vit/projects/cryptoshow-analysis/src/utils')
import eval_utils
import cryptoshow_utils
import vis_utils

CBS_DATA_PATH = '/home/vit/Projects/cryptoshow-analysis/data/A-cluster-ligysis-data/cryptobench-clustered-binding-sites.txt'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/D-visualize/predictions'

# load ground truth binding residues: these have mmcif numbering and need to be mapped to auth labeling
binding_residues_mmcifed, _ = eval_utils.read_test_binding_residues(data_path=CBS_DATA_PATH, pocket_types=['NON_CRYPTIC'])
binding_residues_mmcifed = vis_utils.reformat_binding_residues(binding_residues_mmcifed)

# load predictions
p2rank_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/p2rank', protein_ids=binding_residues_mmcifed.keys())
model_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/finetuning-with-smoothing', protein_ids=binding_residues_mmcifed.keys())

# map binding residues to auth labeling
binding_residues = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in binding_residues_mmcifed.items()}
p2rank_predictions = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in p2rank_predictions_mmcifed.items()}
model_predictions = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in model_predictions_mmcifed.items()}

In [6]:
from pymol.cgo import *

for protein_id in binding_residues.keys():
    this_binding_residues = np.concatenate(binding_residues[protein_id]) if binding_residues[protein_id] else np.array([])
    this_p2rank_predictions = np.concatenate(p2rank_predictions[protein_id]) if p2rank_predictions[protein_id] else np.array([])

    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.color('grey', protein_id)
    if len(this_binding_residues) != 0:
        cmd.color('red', vis_utils.generate_pymol_algebra_selection(protein_id, this_binding_residues))
    else:
        continue
    if len(this_p2rank_predictions) != 0:
        cmd.color('blue', vis_utils.generate_pymol_algebra_selection(protein_id, this_p2rank_predictions))
        intersection = vis_utils.get_intersection(this_binding_residues, this_p2rank_predictions)
        if len(intersection) != 0:
            cmd.color('cyan', vis_utils.generate_pymol_algebra_selection(protein_id, intersection))

    DCCs = vis_utils.compute_DCCs(protein_id, binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id])
    successful_predictions = vis_utils.count_successful_predictions(DCCs)
    print(f'{successful_predictions} / {len(binding_residues_mmcifed[protein_id])} pockets found. {DCCs}')
    
    # cmd.show('surface', protein_id)

    vis_utils.draw_pocket_centers(binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id], protein_id)
    vis_utils.draw_pocket_center_lines(binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id], protein_id)

    cmd.zoom(protein_id)
    cmd.show('surface', protein_id)
    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break

 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
7.389792
11.718629
6.4491515
15.829884
25.996534
24.141903
23.640955
27.444628
20.686243
22.192633
6.314685
8.092655
11.304523
14.360358
24.551104
0 / 3 pockets found. [6.4491515, 20.686243, 6.314685]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
20.552778
5.205196
11.261427
22.807564
0 / 1 pockets found. [5.205196]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
13.154926
14.770398
15.93389
0 / 1 pockets found. [13.154926]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
11.7739525
6.9558115
0 / 2 pockets found. [11.7739525, 6.9558115]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
4.927433
9

In [ ]:
COORDINATES_DIR = '/home/vit/Projects/cryptoshow-analysis/data/A-cluster-ligysis-data/coordinates'

avg_success_pocket_size = []
for protein_id in binding_residues.keys(): 
    this_binding_residues = binding_residues_mmcifed[protein_id]
    this_p2rank_predictions = p2rank_predictions_mmcifed[protein_id]
    sys.path.append('/home/skrhakv/cryptoshow-analysis/src/utils')
    sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/utils')
    import cryptoshow_utils
    
    
    for true_pocket in this_binding_residues:
        dcc = float('inf')
        coordinates = np.load(f'{COORDINATES_DIR}/{protein_id.replace("_", "")}.npy')
        for predicted_pocket in this_p2rank_predictions:
            this_dcc = cryptoshow_utils.compute_center_distance(coordinates, true_pocket, predicted_pocket)
            dcc = min(dcc, this_dcc)
        if dcc < 4.0:
            avg_success_pocket_size.append(len(true_pocket))

np.mean(avg_success_pocket_size)

19.072727272727274

In [17]:
avg_pocket_size = []
protein_id = '4oqoB'
this_binding_residues = binding_residues_mmcifed[protein_id]

for true_pocket in this_binding_residues:
    avg_pocket_size.append(len(true_pocket))
np.median(avg_pocket_size)

6.0

In [18]:
avg_pocket_size


[16, 8, 6, 5, 9, 2, 3]